In [11]:
import numpy as np
from sklearn.datasets import make_classification

In [12]:
from sklearn.datasets import make_classification

X, Y = make_classification(
  n_samples=100, n_features=4, n_redundant=0,
  n_informative=3,  n_clusters_per_class=2, n_classes=3
)

In [49]:
from sklearn.model_selection import train_test_split
import torch 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    test_size=0.25, 
                                                    random_state=42)


X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
Y_train = torch.from_numpy(Y_train).type(torch.LongTensor)
Y_test = torch.from_numpy(Y_test).type(torch.LongTensor)

# Checking the output and input type
print(f"X_train tensor Shape {X_train.shape}, {X_train.dtype}")
print(f"x_test tensor Shape {X_test.shape}, {X_test.dtype}")
print(f"y_train tensor Shape {Y_train.shape}, {Y_train.dtype}")
print(f"y_test tensor Shape {Y_test.shape}, {Y_test.dtype}")

n_unique_labels = len(np.unique(Y_test))
print(f"Unique classes : {n_unique_labels}")

X_train tensor Shape torch.Size([75, 4]), torch.float32
x_test tensor Shape torch.Size([25, 4]), torch.float32
y_train tensor Shape torch.Size([75]), torch.int64
y_test tensor Shape torch.Size([25]), torch.int64
Unique classes : 3


In [50]:
import torch
from torch.utils.data import Dataset, DataLoader

In [72]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.input_features= x
        self.label=y

    def __len__(self):
        return len(self.input_features)
    
    def __getitem__(self, index):
        data= self.input_features[index]
        label_= self.label[index]
        return data, label_

In [188]:
training_data= MyDataset(X_train, Y_train)
testing_data = MyDataset(X_test, Y_test)

batch_size = 8
trainloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
testloader =  DataLoader(testing_data, batch_size=batch_size, shuffle=False)

In [233]:
import torch.nn as nn

input_dim = 4
hidden_layers = 25
output_dim = 3

class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.linear1 = nn.Linear(input_dim, hidden_layers)
    self.linear2 = nn.Linear(hidden_layers, output_dim)
    
  def forward(self, x):
    x = torch.sigmoid(self.linear1(x))
    x = self.linear2(x)
    return x
  
# model and it's parameters 
model= Network()
print(model.parameters)

<bound method Module.parameters of Network(
  (linear1): Linear(in_features=4, out_features=25, bias=True)
  (linear2): Linear(in_features=25, out_features=3, bias=True)
)>


In [234]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [235]:
epochs= 10

for epoch in range(epochs):
    train_loss, test_loss =0,0
    train_count, test_count =0,0
    train_acc, test_acc= 0,0

    model.train()

    for steps,data in enumerate(trainloader):
    
        inputs, labels = data
        outputs= model(inputs)

        #calculate gradients & backword propogation
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()      
        optimizer.step()

        # add the loss 
        train_loss += loss.item()
        train_count += inputs.shape[0]

        _, train_predictions= torch.max(outputs, axis=1)
        train_acc += (train_predictions == labels).sum().item()
      

    with torch.no_grad():
        for test_inputs, test_labels in testloader:
            test_outputs= model(test_inputs)
            test_loss += criterion(test_outputs, test_labels)
            test_count += test_inputs.shape[0]

            _, test_predictions= torch.max(test_outputs, axis=1)
            test_acc += (test_predictions == test_labels).sum().item()

    print(f"Epoch:{epoch} | Train loss: {train_loss/train_count:.4f} | Train accuracy: {train_acc/train_count:.4f} | Test loss: {test_loss/test_count:.4f} | Test accuracy: {test_acc/test_count:.4f}")


Epoch:0 | Train loss: 0.1512 | Train accuracy: 0.1467 | Test loss: 0.1798 | Test accuracy: 0.1600
Epoch:1 | Train loss: 0.1516 | Train accuracy: 0.2000 | Test loss: 0.1795 | Test accuracy: 0.1600
Epoch:2 | Train loss: 0.1521 | Train accuracy: 0.2000 | Test loss: 0.1791 | Test accuracy: 0.1600
Epoch:3 | Train loss: 0.1502 | Train accuracy: 0.1867 | Test loss: 0.1792 | Test accuracy: 0.2000
Epoch:4 | Train loss: 0.1498 | Train accuracy: 0.2000 | Test loss: 0.1790 | Test accuracy: 0.2000
Epoch:5 | Train loss: 0.1495 | Train accuracy: 0.1867 | Test loss: 0.1795 | Test accuracy: 0.1600
Epoch:6 | Train loss: 0.1490 | Train accuracy: 0.2533 | Test loss: 0.1786 | Test accuracy: 0.2000
Epoch:7 | Train loss: 0.1476 | Train accuracy: 0.2933 | Test loss: 0.1780 | Test accuracy: 0.2400
Epoch:8 | Train loss: 0.1478 | Train accuracy: 0.2667 | Test loss: 0.1769 | Test accuracy: 0.2000
Epoch:9 | Train loss: 0.1471 | Train accuracy: 0.2800 | Test loss: 0.1769 | Test accuracy: 0.2400


2

In [225]:
train_predictions, labels

(tensor([0, 1, 1]), tensor([0, 2, 1]))

In [220]:
torch.max(outputs, axis=1)[]

torch.return_types.max(
values=tensor([0.8019, 0.8964, 0.2037], grad_fn=<MaxBackward0>),
indices=tensor([2, 2, 1]))

In [160]:
# epochs= 2

# for epoch in range(epochs):
#     model.train()
#     train_loss= 0
#     test_loss=0
#     total_samples=0
#     correct_predictions= 0

#     for steps,data in enumerate(trainloader):
    
#         inputs, labels = data
#         optimizer.zero_grad()

#         outputs= model(inputs)
#         loss = criterion(outputs, labels)

#         #calculate gradients & backword propogation 
#         loss.backward()
#         optimizer.step() 

#         # add the loss 
#         running_loss_per_epoc += loss.item()

#     with torch.no_grad():
#         for test_inputs, test_labels in testloader:
#             test_outputs = model(test_inputs)
#             test_loss += criterion(test_outputs, test_labels).item()
#             _, predicted = torch.max(test_outputs, 1) 
#             total_samples += test_labels.size(0)
#             correct_predictions += (predicted == test_labels).sum().item()
#             print(test_loss)

#     accuracy = correct_predictions / total_samples
#     avg_test_loss = test_loss / len(testing_data)
#     avg_train_loss = train_loss/ len(training_data)

#     print(f'Epoch:{epoch} | Train loss: {avg_train_loss:.7f} | Test loss: {avg_test_loss:.4f}')

  
  

0.8842911720275879
1.8292537927627563
2.5715860724449158
3.764537751674652
4.555106580257416
5.353415787220001
6.562723219394684
Epoch:0 | Train loss: 0.0000000 | Test loss: 0.2625
0.8371245265007019
1.5569008588790894
2.380488693714142
3.4742533564567566
4.14954000711441
4.925200045108795
6.299934923648834
Epoch:1 | Train loss: 0.0000000 | Test loss: 0.2520


In [159]:
2.0203150510787964/75

0.02693753401438395

In [150]:
len(testloader), len(training_data)

(7, 75)

In [126]:
test_outputs

tensor([[ 1.1078, -1.6956,  0.3433],
        [-0.3032,  0.1201,  0.0611],
        [ 2.0489, -2.9327,  0.6322],
        [-1.3448,  1.7912, -0.6751]])

In [132]:
torch.max(test_outputs, dim=1)

torch.return_types.max(
values=tensor([1.1078, 0.1201, 2.0489, 1.7912]),
indices=tensor([0, 1, 0, 1]))

In [133]:
torch.argmax(test_outputs, dim=1)

tensor([0, 1, 0, 1])

In [135]:
test_labels

tensor([2, 0, 0, 1])

In [136]:
(torch.argmax(test_outputs, dim=1) == test_labels).sum().item()

2